<a href="https://www.kaggle.com/code/nataliasz/quick-lgbm-no-leak?scriptVersionId=97453334" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# JPX Tokyo Stock Exchange Prediction <span style="color:DarkCyan"> with LGBM</span>

Thank you for viewing my notebook, I hope you enjoy it 📊<br>
Don't hesitate to leave any feedback 😉

# Table of Contents
1. [Overview](#Overview)
1. [Load JPX data](#Load-JPX-data)
2. [Preprocess](#Preprocess)
3. [Train the LGBM Model](#Train-the-LGBM-Model)
4. [Predict Test Data](#Predict-Test-Data)
5. [Submit](#Submit)

# Overview

In this notebook, I will build a <span style="color:DarkCyan">Light Gradient Boosting Model</span> for the [JPX Tokyo Stock Exchange Prediction Competition](https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction).

### Quick introduction to <span style="color:DarkCyan">LGBM</span>

[Light GBM](https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc) is a gradient boosting model that uses decision tree algorithm.<br>
<span style="color:DarkCyan">Light GBM</span> grows tree vertically while other gradient boosting algorithms (e.g. XGBoost) grow trees horizontally.<br>
<span style="color:DarkCyan">LGBM</span> chooses the leaf with max delta loss to grow. Holding leaf fixed, leaf-wise algorithms tend to achieve lower loss than level-wise algorithms.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import jpx_tokyo_market_prediction

pd.set_option('display.max_columns', 100)

In [2]:
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # gbdt - traditional Gradient Boosting Decision Tree
    'objective': 'regression',  # L2 loss
    'metric': 'rmse',
    'learning_rate': 0.05,
    'lambda_l1': 0.5,  # L1 regularization
    'lambda_l2': 0.5,  # L2 regularization
    'num_leaves': 10,
    'feature_fraction': 0.5,  # LightGBM will select 50% of features before training each tree
    'bagging_fraction': 0.5,  # LightGBM will select 50% part of data without resampling
    'bagging_freq': 5,  #  perform bagging at every k iteration
    'min_child_samples': 10,
    'seed': 42
}

Load datasets

# Load JPX data

In [3]:
file_path = '/kaggle/input/jpx-tokyo-stock-exchange-prediction/'
prices = pd.read_csv(Path(file_path, 'train_files/stock_prices.csv'))
stock_list = pd.read_csv(Path(file_path, 'stock_list.csv'))

In [4]:
prices.describe()

,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,Target
count,2.332531e+06,2.324923e+06,2.324923e+06,2.324923e+06,2.324923e+06,2.332531e+06,2.332531e+06,18865.000000,2.332293e+06
mean,5.894835e+03,2.594511e+03,2.626540e+03,2.561227e+03,2.594023e+03,6.919366e+05,1.000508e+00,22.017730,4.450962e-04
std,2.404161e+03,3.577192e+03,3.619363e+03,3.533494e+03,3.576538e+03,3.911256e+06,6.773040e-02,29.882453,2.339879e-02
min,1.301000e+03,1.400000e+01,1.500000e+01,1.300000e+01,1.400000e+01,0.000000e+00,1.000000e-01,0.000000,-5.785414e-01
25%,3.891000e+03,1.022000e+03,1.035000e+03,1.009000e+03,1.022000e+03,3.030000e+04,1.000000e+00,5.000000,-1.049869e-02
50%,6.238000e+03,1.812000e+03,1.834000e+03,1.790000e+03,1.811000e+03,1.071000e+05,1.000000e+00,15.000000,0.000000e+00
75%,7.965000e+03,3.030000e+03,3.070000e+03,2.995000e+03,3.030000e+03,4.021000e+05,1.000000e+00,30.000000,1.053159e-02
max,9.997000e+03,1.099500e+05,1.105000e+05,1.072000e+05,1.095500e+05,6.436540e+08,2.000000e+01,1070.000000,1.119512e+00


In [5]:
prices.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


Display information about main dataset

In [6]:
prices.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332531 entries, 0 to 2332530
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   RowId             2332531 non-null  object 
 1   Date              2332531 non-null  object 
 2   SecuritiesCode    2332531 non-null  int64  
 3   Open              2324923 non-null  float64
 4   High              2324923 non-null  float64
 5   Low               2324923 non-null  float64
 6   Close             2324923 non-null  float64
 7   Volume            2332531 non-null  int64  
 8   AdjustmentFactor  2332531 non-null  float64
 9   ExpectedDividend  18865 non-null    float64
 10  SupervisionFlag   2332531 non-null  bool   
 11  Target            2332293 non-null  float64
dtypes: bool(1), float64(7), int64(2), object(2)
memory usage: 198.0+ MB


# Preprocess

Manage data id: add variable describing how many days have passed from the beginning of the dataset 

In [7]:
prices['Date'] = pd.to_datetime(prices['Date'])
min_date = prices['Date'].min()
prices['date_rank'] = (prices['Date'] - min_date).dt.days

Choose features to use as independent variables in the model

In [8]:
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'date_rank', 'SecuritiesCode']

In [9]:
prices = prices.dropna(subset=features)

In [10]:
target = prices.pop('Target')
# scaler = StandardScaler()
# target = scaler.fit_transform(np.array(target).reshape(-1, 1)).ravel()
# target = pd.Series(target, index = prices.index)
target_mean = target.mean()

In [11]:
target.describe()

count    2.324923e+06
mean     4.262053e-04
std      2.339197e-02
min     -5.785414e-01
25%     -1.052632e-02
50%      0.000000e+00
75%      1.052632e-02
max      6.182380e-01
Name: Target, dtype: float64

In [12]:
train_f, valid_f = train_test_split(prices[features], test_size=0.2, shuffle=False)
train_idx = train_f.index
valid_idx = valid_f.index
lgb_train = lgb.Dataset(train_f, target[train_idx])
lgb_valid = lgb.Dataset(valid_f, target[valid_idx], reference=lgb_train)

In [13]:
train_f.head()

,Open,High,Low,Close,Volume,date_rank,SecuritiesCode
0,2734.0,2755.0,2730.0,2742.0,31400,0,1301
1,568.0,576.0,563.0,571.0,2798500,0,1332
2,3150.0,3210.0,3140.0,3210.0,270800,0,1333
3,1510.0,1550.0,1510.0,1550.0,11300,0,1376
4,3270.0,3350.0,3270.0,3330.0,150800,0,1377


# Train the <span style="color:DarkCyan">LGBM</span> Model

In [14]:
model = lgb.train(
    lgbm_params,
    lgb_train,
    valid_sets=[lgb_train, lgb_valid],
    valid_names=['Train', 'Valid'],
    num_boost_round=2000,
    early_stopping_rounds=100,
    verbose_eval=100,
)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.074242 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 1859938, number of used features: 7
[LightGBM] [Info] Start training from score 0.000384
Training until validation scores don't improve for 100 rounds
[100]	Train's rmse: 0.0231328	Valid's rmse: 0.0219297
Early stopping, best iteration is:
[25]	Train's rmse: 0.0233947	Valid's rmse: 0.0219286


# Predict Test Data

In [15]:
test_prices = pd.read_csv(Path(file_path, 'example_test_files/stock_prices.csv'))
test_prices['date_rank'] = (pd.to_datetime(test_prices['Date']) - min_date).dt.days

In [16]:
preds =  model.predict(test_prices[features], num_iteration=model.best_iteration)
preds

array([0.0005377 , 0.00071663, 0.0005377 , ..., 0.00050657, 0.00049588,
       0.0005898 ])

In [17]:
pd.Series(preds).fillna(target_mean).rank(ascending = False,method = 'first').astype(int)

0       2068
1        197
2       2069
3        671
4       1360
        ... 
3995     528
3996     529
3997    2986
3998    3090
3999     972
Length: 4000, dtype: int64

# Submit

In [18]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    prices['date_rank'] = (pd.to_datetime(prices['Date']) - min_date).dt.days
    preds = model.predict(prices[features], num_iteration=model.best_iteration)
    preds = np.squeeze(preds)
    print(preds)
    sample_prediction["Prediction"] = preds
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
[0.0005377  0.00071663 0.0005377  ... 0.00050657 0.00049588 0.00056692]
[0.0005377  0.00071663 0.0005377  ... 0.00050657 0.00049588 0.0005898 ]
